In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q -U einops
!pip install -q -U safetensors
!pip install -q -U torch
!pip install -q -U xformers
!pip install -q -U datasets

In [1]:
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          BitsAndBytesConfig)
import transformers
import torch
from torch.utils.data import Dataset
import torch
from transformers import AutoTokenizer
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from huggingface_hub import notebook_login


In [3]:
notebook_login()

In [4]:
class TextDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx])
                for key, val in self.encodings.items()}
        item["labels"] = item["input_ids"].clone()
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [ ]:
def run_inference(
        model, tokenizer,
        question='<s> [INST] <<SYS>> Answer this question <</SYS>> '+'est il droppy la meilleure solution?'+ ' [/INST] '):
    model.config.use_cache = True
    model.eval()
    print("Question:")
    print(question)
    encoding = tokenizer(question, return_tensors="pt").to("cuda:0")
    with torch.inference_mode():
        output = model.generate(input_ids=encoding.input_ids,
                                attention_mask=encoding.attention_mask,
                                max_new_tokens=200, do_sample=True,
                                temperature=0.000001,pad_token_id=tokenizer.eos_token_id,
                                eos_token_id=tokenizer.eos_token_id, top_k=0)
        print("Answer:")
        print(tokenizer.decode(output[0], skip_special_tokens=True))

In [6]:
def load_pretrained_model(model_id):


    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_id, device_map="auto",bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
        trust_remote_code=True)


    return model, tokenizer



In [7]:
def prepare_dataset(tokenizer, dataset_file):

    data = load_dataset("json", data_files=dataset_file,
                        field="data")

    train_dataset = data['train'].select(range(63)).map(
        lambda x: {"input_text": '<s> [INST] <<SYS>> Answer this question <</SYS>> '+x['question'] + ' [/INST] '+x['answer']+" </s>"})

    train_encodings = tokenizer(
        train_dataset['input_text'],truncation=True, padding=True,max_length=400,
         return_tensors='pt')

    train_dataset = TextDataset(train_encodings)

    return train_dataset


In [8]:
def launch_training(model, tokenizer, train_dataset):
    training_configuration = transformers.TrainingArguments(
            num_train_epochs=50,
            per_device_train_batch_size=6,
            gradient_accumulation_steps=3,
            warmup_ratio=0.05,

            save_steps = 900,
            logging_steps = 10,
            learning_rate=1e-4,
            fp16=True,

            output_dir="outputs",
            optim="paged_adamw_8bit",
            lr_scheduler_type='constant',
        )
    trainer = transformers.Trainer(
        model=model,
        train_dataset=train_dataset,
        args=training_configuration,
        data_collator=transformers.DataCollatorForLanguageModeling(
            tokenizer, mlm=False),
    )
    model.config.use_cache = False
    trainer.train()
    MODEL = "zaroui/vicuna13B"

    model.push_to_hub(
        MODEL, use_auth_token=True
    )

    tokenizer.push_to_hub(
        MODEL, use_auth_token=True
    )
    

In [9]:
dataset_file="/kaggle/input/datafinale/questionETreponse.json"
model, tokenizer = load_pretrained_model("hiepnh/vicuna-13B-1.1-HF-sharded")

ValueError: The current `device_map` had weights offloaded to the disk. Please provide an `offload_folder` for them. Alternatively, make sure you have `safetensors` installed if the model you are using offers the weights in this format.

In [ ]:
train_dataset = prepare_dataset(tokenizer, dataset_file)


In [ ]:
launch_training(model, tokenizer, train_dataset)


In [ ]:
run_inference(model, tokenizer)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 1.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


OSError: zaroui/vicunabest does not appear to have a file named config.json. Checkout 'https://huggingface.co/zaroui/vicunabest/main' for available files.

{'perplexities': [1360.273681640625], 'mean_perplexity': 1360.273681640625}